In [92]:
# Import the libraries
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

In [93]:
# Declare the credentials
client_id = '2a7adde1b07a4bf9a5fd750d6ff6f28d'
client_secret = '74a5c5859a0548ccae791419cc29c8fa'
redirect_uri='http://localhost:7777/callback'
username = 'ethan.falcao'

In [94]:
# Authorization flow
scope = 'user-top-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [95]:
# # Getting playlist IDs from each of Spotify's playlists
playlists = sp.user_playlists('spotify')
spotify_playlist_ids = []
while playlists:
    for i, playlist in enumerate(playlists['items']):
        spotify_playlist_ids.append(playlist['uri'][-22:])
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None
spotify_playlist_ids[:20]
#change 5 to 20

['37i9dQZF1DXcBWIGoYBM5M',
 '37i9dQZF1DX0XUsuxWHRQd',
 '37i9dQZF1DX1lVhptIYRda',
 '37i9dQZF1DX10zKzsJ2jva',
 '37i9dQZF1DX4JAvHpjipBk',
 '37i9dQZF1DX4sWSpwq3LiO',
 '37i9dQZF1DX4SBhb3fqCJd',
 '37i9dQZF1DWXRqgorJj26U',
 '37i9dQZF1DX4dyzvuaRJ0n',
 '37i9dQZF1DXcF6B6QPhFDv',
 '37i9dQZF1DWXJfnUiYjUKT',
 '37i9dQZF1DXcRXFNfZr7Tp',
 '37i9dQZF1DX4o1oenSJRJd',
 '37i9dQZF1DXbTxeAdrVG2l',
 '37i9dQZF1DX4UtSsGT1Sbe',
 '37i9dQZF1DWTJ7xPn4vNaz',
 '37i9dQZF1DXaKIA8E7WcJj',
 '37i9dQZF1DWSV3Tk4GO2fq',
 '37i9dQZF1DWTwnEm1IYyoj',
 '37i9dQZF1DX2A29LI7xHn1']

In [96]:
len(spotify_playlist_ids)

1108

In [97]:
#Getting tracks from Spotify playlists
# Creating a function to get the first 50 tracks IDs from a playlist
def getTrackIDs(playlist_id):
    try:
        playlist = sp.playlist(playlist_id)
        for item in playlist['tracks']['items'][:100]:
            track = item['track']
            ids.append(track['id'])
    except Exception as e:
        print(f"Error fetching playlist {playlist_id}: {e}")
#change 100 to 50

In [98]:
# %%time
# # Gathering track ids
ids = []
for x in spotify_playlist_ids[:210]:
    getTrackIDs(x)
ids[:5]
#change 50 to 200

Error fetching playlist 37i9dQZF1DXc0fQ4sGOnVJ: 'NoneType' object is not subscriptable
Error fetching playlist 37i9dQZF1DXcZDD7cfEKhW: 'NoneType' object is not subscriptable
Error fetching playlist 37i9dQZF1DXa41CMuUARjl: 'NoneType' object is not subscriptable
Error fetching playlist 37i9dQZF1DX5GiUwx1edLZ: 'NoneType' object is not subscriptable
Error fetching playlist 37i9dQZF1DX5Q27plkaOQ3: 'NoneType' object is not subscriptable
Error fetching playlist 37i9dQZF1DWV0gynK7G6pD: 'NoneType' object is not subscriptable
Error fetching playlist 37i9dQZF1DWYHkSks5rujC: 'NoneType' object is not subscriptable
Error fetching playlist 37i9dQZF1DWZreqadA03A8: 'NoneType' object is not subscriptable
Error fetching playlist 37i9dQZF1DXdwF9DytRe8w: 'NoneType' object is not subscriptable


['3rUGC1vUpkDG9CZFHMur1t',
 '7aqfrAY2p9BUSiupwk3svU',
 '7dJYggqjKo71KI9sLzqCs8',
 '4MjDJD8cW7iVeWInc2Bdyj',
 '5aIVCx5tnk0ntmdiinnYvw']

In [99]:
len(ids)

15045

In [100]:
#don't run  

In [101]:
def fetch_audio_features(sp, track_ids):
    # Make the API request
    audio_features = []
    index = 0
    while index < len(track_ids):
        audio_features += sp.audio_features(track_ids[index:index + 70])
        index += 70
    
    # Create an empty list to feed in different characteristics of the tracks
    features_list = []
        
    for track_id, features in zip(track_ids, audio_features):
        features_list.append([
        track_id,
        features['time_signature'], 
        features['danceability'],
        features['acousticness'],
        features['energy'],
        features['tempo'],
        features['instrumentalness'],
        features['loudness'],
        features['liveness'],
        features['duration_ms'],
        features['key'],
        features['valence'],
        features['speechiness'],
        features['mode']
    ])
        


    df_audio_features = pd.DataFrame(features_list, columns=['track_id','time_signature','danceability','acousticness', 'energy', 'tempo', 'instrumentalness', 'loudness', 
                                                             'liveness', 'duration_ms', 'key', 'valence',  'speechiness', 'mode'])

    
    return df_audio_features

In [102]:
ids_features =  fetch_audio_features(sp, ids)
ids_features.head()

,track_id,time_signature,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness,mode
0,3rUGC1vUpkDG9CZFHMur1t,1,0.750,0.2560,0.733,111.018,0.000000,-3.180,0.114,131872,6,0.844,0.0319,0
1,7aqfrAY2p9BUSiupwk3svU,4,0.470,0.0277,0.640,163.920,0.000000,-7.779,0.382,247444,2,0.248,0.3200,1
2,7dJYggqjKo71KI9sLzqCs8,4,0.750,0.2280,0.674,123.026,0.000089,-6.128,0.122,265360,8,0.392,0.0970,0
3,4MjDJD8cW7iVeWInc2Bdyj,4,0.787,0.1500,0.621,139.056,0.000402,-5.009,0.580,267194,4,0.130,0.0680,0
4,5aIVCx5tnk0ntmdiinnYvw,4,0.673,0.0856,0.722,117.187,0.000000,-3.495,0.137,200256,3,0.519,0.0755,0


In [108]:
#Export to csv
ids_features.to_csv('data/data_extraction/15k_features_ids.csv', index=False)

In [ ]:
#dont run below

In [ ]:
# Due to Rate limiting Split the list into smaller batches of 2,000 IDs each
batch_size = 2000
ids_lists = [ids[i:i + batch_size] for i in range(0, len(ids), batch_size)]

# Iterate throught the list of list and get the feature of each song id 
song_info = []
# Iterate through each list of song IDs
for song_id_list in ids_lists:
    for song_id in song_id_list:
        try:
            # Search for the track using its ID
            track = sp.track(song_id)
            song_name = track['name']
            # Get the artists as a list of dictionaries and extract their names
            artists = [artist['name'] for artist in track['artists']]
            song_info.append({'ID': song_id, 'Song Name': song_name, 'Artists': artists})
        except:
            time.sleep(0.5)
            song_info.append({'ID': song_id, 'Song Name': 'Not Found'})
            song_id_list.remove(song_id)  # Remove the song ID from the current list
            pass
# Create a DataFrame from the collected song information
song_df = pd.DataFrame(song_info)


In [ ]:
#dont run above

In [17]:
#song_ids = ids[:5000]
#song_ids = ids[5000:10000]
song_ids = ids[10000:]

In [18]:
song_info = []

for song_id in song_ids:
    try:
        # Search for the track using its ID
        track = sp.track(song_id)
        song_name = track['name']
        # Get the album name
        album_name = track['album']['name']
        # Get the artists a
        artists = [artist['name'] for artist in track['artists']]
        popularity = track['popularity']
        song_info.append({'ID': song_id, 
                          'Song Name': song_name, 
                          'Artists': artists, 
                          'Album Name': album_name, 
                          'Popularity': popularity})
    except:
        time.sleep(.5)
        song_info.append({'ID': song_id, 'Song Name': 'Not Found'})
        song_ids.remove(song_id)
        pass

song_df = pd.DataFrame(song_info)
song_df    

,ID,Song Name,Artists,Album Name,Popularity
0,6q41rWOFtGs0K63lm0tAPa,Simply Won't Do,[Disclosure],Alchemy,56
1,3srre2C7GNHuowM46tOe1J,On My Mind,[Nic Fanciulli],On My Mind,35
2,0HfwmVJGd0kzoVXLB6g7cX,WTF,"[Dale Howard, Roland Clark]",WTF,43
3,6As1zfxQHAgeQPBK2paaqh,The Greatest Thing Alive,"[Mark Knight, Green Velvet, James Hurr]",The Greatest Thing Alive,47
4,5LZNaPmgnxH2YHwmU9G0GL,Its Time to Move,[Sosa UK],Its Time to Move,51
...,...,...,...,...,...
4873,6ZSTinOwx5dKFYc6iYyGDn,Leaving on a Jet Plane,"[Peter, Paul and Mary]",Album 1700,62
4874,4DKRLDVRIGojztAOEdvmXK,I Put A Spell On You,[Nina Simone],The Best Of Nina Simone,0
4875,0OwvrdjaJOypNsNQHyBWE5,California Soul,[Marlena Shaw],The Spice Of Life,69
4876,3IKyTBHZSgCx2v1ZX8v0TZ,Albatross - 2018 Remaster,[Fleetwood Mac],The Pious Bird of Good Omen,0


In [19]:
song_df.to_csv('/Users/ethan/Documents/Spotify_recommendations/Github_pull/github/Spotify_Reccommendation_Algorithm/15k_info.csv', index=False)

In [105]:
df_5k=pd.read_csv('data/data_extraction/5k_info.csv', sep=",")
df_10k=pd.read_csv('data/data_extraction/10k_info.csv', sep=",")
df_15k=pd.read_csv('data/data_extraction/15k_info.csv', sep=",")
df_features=pd.read_csv('data/data_extraction/15k_features_ids.csv', sep=",")
#df_features  = ids_features

In [115]:
df_info = pd.concat([df_5k, df_10k, df_15k])
df_info = df_info.rename(columns={'ID': 'track_id',
                                  'Song Name': 'Song',
                                  'Album Name': 'Album'  })
df_features = df_features.rename(columns={
                                  'duration_ms': 'length'                           
                                  })
df_info.head()

,track_id,Song,Artists,Album,Popularity
0,3rUGC1vUpkDG9CZFHMur1t,greedy,['Tate McRae'],greedy,98
1,7aqfrAY2p9BUSiupwk3svU,First Person Shooter (feat. J. Cole),"['Drake', 'J. Cole']",For All The Dogs,91
2,7dJYggqjKo71KI9sLzqCs8,Agora Hills,['Doja Cat'],Scarlet,89
3,4MjDJD8cW7iVeWInc2Bdyj,MONACO,['Bad Bunny'],nadie sabe lo que va a pasar mañana,94
4,5aIVCx5tnk0ntmdiinnYvw,Water,['Tyla'],Water,92


In [90]:
df_features = df_features[['track_id','time_signature','danceability','acousticness', 'energy', 'tempo', 'instrumentalness', 'loudness', 
                                                             'liveness', 'length', 'key', 'valence',  'speechiness', 'mode']]

In [117]:
df = pd.merge(df_features, df_info, on='track_id')
df = df.reindex(columns=['track_id', 'Song', 'Artists', 'Album', 'length', 'time_signature', 
                         'danceability', 'acousticness', 'energy', 'tempo', 'instrumentalness', 
                         'loudness', 'liveness', 'key', 'valence', 'speechiness', 'mode', 'Popularity'])

df = df.drop_duplicates()   
df = df.dropna()
df.head()

,track_id,Song,Artists,Album,length,time_signature,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,key,valence,speechiness,mode,Popularity
0,3rUGC1vUpkDG9CZFHMur1t,greedy,['Tate McRae'],greedy,131872,1,0.750,0.2560,0.733,111.018,0.000000,-3.180,0.114,6,0.844,0.0319,0,98
4,7aqfrAY2p9BUSiupwk3svU,First Person Shooter (feat. J. Cole),"['Drake', 'J. Cole']",For All The Dogs,247444,4,0.470,0.0277,0.640,163.920,0.000000,-7.779,0.382,2,0.248,0.3200,1,91
20,7dJYggqjKo71KI9sLzqCs8,Agora Hills,['Doja Cat'],Scarlet,265360,4,0.750,0.2280,0.674,123.026,0.000089,-6.128,0.122,8,0.392,0.0970,0,89
29,4MjDJD8cW7iVeWInc2Bdyj,MONACO,['Bad Bunny'],nadie sabe lo que va a pasar mañana,267194,4,0.787,0.1500,0.621,139.056,0.000402,-5.009,0.580,4,0.130,0.0680,0,94
38,5aIVCx5tnk0ntmdiinnYvw,Water,['Tyla'],Water,200256,4,0.673,0.0856,0.722,117.187,0.000000,-3.495,0.137,3,0.519,0.0755,0,92


In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12476 entries, 0 to 20355
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          12476 non-null  object 
 1   Song              12476 non-null  object 
 2   Artists           12476 non-null  object 
 3   Album             12476 non-null  object 
 4   length            12476 non-null  int64  
 5   time_signature    12476 non-null  int64  
 6   danceability      12476 non-null  float64
 7   acousticness      12476 non-null  float64
 8   energy            12476 non-null  float64
 9   tempo             12476 non-null  float64
 10  instrumentalness  12476 non-null  float64
 11  loudness          12476 non-null  float64
 12  liveness          12476 non-null  float64
 13  key               12476 non-null  int64  
 14  valence           12476 non-null  float64
 15  speechiness       12476 non-null  float64
 16  mode              12476 non-null  int64  
 17

## Top 50 tracks from user


In [135]:
# Creating a function get features of each track from track id
def getTrackFeatures(track_id):
  meta = sp.track(track_id)
  features = sp.audio_features(track_id)

  # meta
  track_id = track_id
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  mode= features[0]['mode']
  key = features[0]['key']
  valence = features[0]['valence']

  track = [track_id, name, album, artist, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature, mode, key, valence]
  return track

In [136]:
def create_track_dataframe(results):
    track_name = []
    track_id = []
    artist = []
    album = []
    duration = []
    popularity = []
    
    for i, items in enumerate(results['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])

    df_favourite = pd.DataFrame({
        "track_name": track_name,
        "album": album,
        "track_id": track_id,
        "artist": artist,
        "duration": duration,
        "popularity": popularity
    })

    return df_favourite

In [137]:
# Getting top 50 tracks from user
short = create_track_dataframe(sp.current_user_top_tracks(limit=1000, offset=0,time_range='short_term'))
medium = create_track_dataframe(sp.current_user_top_tracks(limit=1000, offset=0,time_range='medium_term'))
long = create_track_dataframe(sp.current_user_top_tracks(limit=1000, offset=0,time_range='long_term'))

In [138]:
short.head()

,track_name,album,track_id,artist,duration,popularity
0,Flight's Booked,"Honestly, Nevermind",6Yj7Zhxt73uvwFFvzQXdxO,Drake,254407,68
1,Away From Home,For All The Dogs,4IcbUXrAzBtn6iUO1LP471,Drake,258592,81
2,Another Late Night (feat. Lil Yachty),For All The Dogs,17IBDpAl3J39swaoPlfGpp,Drake,170414,83
3,Trojan Horse,Split Decision,2HALQBSAvpw1oCzL5QsrT2,Dave,248866,76
4,Sprinter,Sprinter,2FDTHlrBguDzQkp7PVj16Q,Dave,229133,94


In [139]:
df.head()

,track_id,Song,Artists,Album,length,time_signature,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,key,valence,speechiness,mode,Popularity,favorite
0,3rUGC1vUpkDG9CZFHMur1t,greedy,['Tate McRae'],greedy,131872,1,0.750,0.2560,0.733,111.018,0.000000,-3.180,0.114,6,0.844,0.0319,0,98,0
4,7aqfrAY2p9BUSiupwk3svU,First Person Shooter (feat. J. Cole),"['Drake', 'J. Cole']",For All The Dogs,247444,4,0.470,0.0277,0.640,163.920,0.000000,-7.779,0.382,2,0.248,0.3200,1,91,0
20,7dJYggqjKo71KI9sLzqCs8,Agora Hills,['Doja Cat'],Scarlet,265360,4,0.750,0.2280,0.674,123.026,0.000089,-6.128,0.122,8,0.392,0.0970,0,89,0
29,4MjDJD8cW7iVeWInc2Bdyj,MONACO,['Bad Bunny'],nadie sabe lo que va a pasar mañana,267194,4,0.787,0.1500,0.621,139.056,0.000402,-5.009,0.580,4,0.130,0.0680,0,94,0
38,5aIVCx5tnk0ntmdiinnYvw,Water,['Tyla'],Water,200256,4,0.673,0.0856,0.722,117.187,0.000000,-3.495,0.137,3,0.519,0.0755,0,92,0


In [140]:
%%time
def user_top_tracks(df):
    fav_tracks = []
    for track_id in df['track_id']:
        try:
            track = getTrackFeatures(track_id)  # Assuming getTrackFeatures is a function that returns track features
            fav_tracks.append(track)
        except:
            pass
    fav_tracks = pd.DataFrame(fav_tracks, columns = ['track_id', 'Song', 'Artists', 'Album', 'length', 'time_signature',
       'danceability', 'acousticness', 'energy', 'tempo', 'instrumentalness',
       'loudness', 'liveness', 'key', 'valence', 'speechiness', 'mode',
       'Popularity'])
  
    return pd.DataFrame(fav_tracks)  # Convert the list of track features to a dataframe

short_term = user_top_tracks(short)
medium_term = user_top_tracks(medium)
long_term = user_top_tracks(long)

CPU times: total: 594 ms
Wall time: 34.5 s


In [143]:
short_term.head()

,track_id,Song,Artists,Album,length,time_signature,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,key,valence,speechiness,mode,Popularity
0,6Yj7Zhxt73uvwFFvzQXdxO,Flight's Booked,"Honestly, Nevermind",Drake,254407,68,0.593,0.8860,0.410,0.0993,0.1150,-13.097,0.0531,122.584,4,0,3,0.201
1,4IcbUXrAzBtn6iUO1LP471,Away From Home,For All The Dogs,Drake,258592,81,0.848,0.0195,0.625,0.0000,0.1090,-7.640,0.0587,93.019,4,1,1,0.226
2,17IBDpAl3J39swaoPlfGpp,Another Late Night (feat. Lil Yachty),For All The Dogs,Drake,170414,83,0.684,0.0347,0.541,0.0000,0.1380,-8.040,0.0483,158.046,4,0,5,0.214
3,2HALQBSAvpw1oCzL5QsrT2,Trojan Horse,Split Decision,Dave,248866,76,0.891,0.4750,0.711,0.0000,0.2270,-4.429,0.3320,138.310,4,1,7,0.743
4,2FDTHlrBguDzQkp7PVj16Q,Sprinter,Sprinter,Dave,229133,94,0.916,0.1860,0.583,0.0000,0.0798,-8.067,0.2410,141.095,4,1,1,0.661


In [144]:
df_fav = pd.concat([short_term, medium_term, long_term])
df_fav = df_fav.drop_duplicates()

In [145]:
df_fav.head()

,track_id,Song,Artists,Album,length,time_signature,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,key,valence,speechiness,mode,Popularity
0,6Yj7Zhxt73uvwFFvzQXdxO,Flight's Booked,"Honestly, Nevermind",Drake,254407,68,0.593,0.8860,0.410,0.0993,0.1150,-13.097,0.0531,122.584,4,0,3,0.201
1,4IcbUXrAzBtn6iUO1LP471,Away From Home,For All The Dogs,Drake,258592,81,0.848,0.0195,0.625,0.0000,0.1090,-7.640,0.0587,93.019,4,1,1,0.226
2,17IBDpAl3J39swaoPlfGpp,Another Late Night (feat. Lil Yachty),For All The Dogs,Drake,170414,83,0.684,0.0347,0.541,0.0000,0.1380,-8.040,0.0483,158.046,4,0,5,0.214
3,2HALQBSAvpw1oCzL5QsrT2,Trojan Horse,Split Decision,Dave,248866,76,0.891,0.4750,0.711,0.0000,0.2270,-4.429,0.3320,138.310,4,1,7,0.743
4,2FDTHlrBguDzQkp7PVj16Q,Sprinter,Sprinter,Dave,229133,94,0.916,0.1860,0.583,0.0000,0.0798,-8.067,0.2410,141.095,4,1,1,0.661


In [150]:
df_fav.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118 entries, 0 to 49
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          118 non-null    object 
 1   Song              118 non-null    object 
 2   Artists           118 non-null    object 
 3   Album             118 non-null    object 
 4   length            118 non-null    int64  
 5   time_signature    118 non-null    int64  
 6   danceability      118 non-null    float64
 7   acousticness      118 non-null    float64
 8   energy            118 non-null    float64
 9   tempo             118 non-null    float64
 10  instrumentalness  118 non-null    float64
 11  loudness          118 non-null    float64
 12  liveness          118 non-null    float64
 13  key               118 non-null    float64
 14  valence           118 non-null    int64  
 15  speechiness       118 non-null    int64  
 16  mode              118 non-null    int64  
 17  Pop

In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12476 entries, 0 to 20355
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          12476 non-null  object 
 1   Song              12476 non-null  object 
 2   Artists           12476 non-null  object 
 3   Album             12476 non-null  object 
 4   length            12476 non-null  int64  
 5   time_signature    12476 non-null  int64  
 6   danceability      12476 non-null  float64
 7   acousticness      12476 non-null  float64
 8   energy            12476 non-null  float64
 9   tempo             12476 non-null  float64
 10  instrumentalness  12476 non-null  float64
 11  loudness          12476 non-null  float64
 12  liveness          12476 non-null  float64
 13  key               12476 non-null  int64  
 14  valence           12476 non-null  float64
 15  speechiness       12476 non-null  float64
 16  mode              12476 non-null  int64  
 17

In [125]:
print(df.shape)
print(df_fav.shape)

(12476, 18)
(118, 18)


In [149]:
df_fav['favorite'] = 1
df['favorite'] = 0 

In [151]:
# Combining the favorite dataframe with the dataframe of Spotify's songs
combined = pd.concat([df, df_fav])
combined.shape

(12594, 19)

In [152]:
combined.favorite.value_counts()

favorite
0    12476
1      118
Name: count, dtype: int64

In [159]:
df_fav = combined.loc[combined['favorite'] == 1]
df_fav.head()

,track_id,Song,Artists,Album,length,time_signature,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,key,valence,speechiness,mode,Popularity,favorite
0,6Yj7Zhxt73uvwFFvzQXdxO,Flight's Booked,"Honestly, Nevermind",Drake,254407,68,0.593,0.8860,0.410,0.0993,0.1150,-13.097,0.0531,122.584,4.0,0.0,3,0.201,1
1,4IcbUXrAzBtn6iUO1LP471,Away From Home,For All The Dogs,Drake,258592,81,0.848,0.0195,0.625,0.0000,0.1090,-7.640,0.0587,93.019,4.0,1.0,1,0.226,1
2,17IBDpAl3J39swaoPlfGpp,Another Late Night (feat. Lil Yachty),For All The Dogs,Drake,170414,83,0.684,0.0347,0.541,0.0000,0.1380,-8.040,0.0483,158.046,4.0,0.0,5,0.214,1
3,2HALQBSAvpw1oCzL5QsrT2,Trojan Horse,Split Decision,Dave,248866,76,0.891,0.4750,0.711,0.0000,0.2270,-4.429,0.3320,138.310,4.0,1.0,7,0.743,1
4,2FDTHlrBguDzQkp7PVj16Q,Sprinter,Sprinter,Dave,229133,94,0.916,0.1860,0.583,0.0000,0.0798,-8.067,0.2410,141.095,4.0,1.0,1,0.661,1


In [158]:
df = combined.loc[combined['favorite'] != 1]

In [160]:
print(df.shape)
print(df_fav.shape)

(12476, 19)
(118, 19)


Export to csv

In [161]:
df.to_csv('data/df.csv', index=False)
df_fav.to_csv('data/df_fav.csv', index=False)